# produce patterns to be labeled

In [122]:
import src.osm.osmdb_filter; reload(src.osm.osmdb_filter)
from src.osm.osmdb_filter import select_bk_facs
import src.constants; reload(src.constants)
from src.constants import fn_osm_db_dc, bk_type

In [111]:
def have_tag_value(obj, tag='*', value='*'):
    if not obj['tags']:  # have no tag, discard it whatever query is
        return False
    if tag == '*':  # True for having any tag
        return True
    if not tag in obj['tags']:
        return False
    if value == '*':
        return True
    return obj['tags'][tag] in value

def and_filter(obj, and_tag):
    for tag, value in and_tag:
        if tag.startswith('-'):
            if have_tag_value(obj, tag[1:], value):
                return False
        else:
            if not have_tag_value(obj, tag, value):
                return False
    return True    

In [108]:
df = select_bk_facs('data/'+fn_osm_db_dc, df=True, ot='W')

bk_ways = []
for (ot, oid), grp in df.groupby(['ot','oid']):
    tags = {key:value for key, value in grp[['key','value']].values}
    way = {'id':oid, 'tags':tags}
    bk_ways.append(way)


bk_tps = bk_type.keys()
result = []
for way in bk_ways:
    result.append((way['id'], ','.join([tp for tp in bk_tps if and_filter(way,bk_type[tp])  ])))

df = pd.DataFrame(result, columns=['id','type'])

p = []
for pattern, grp in df.merge(vectors[['id','pattern']]).groupby('pattern'):
    p.append( (pattern, '##'.join(list(set(grp['type'].values))), grp.shape[0]))
pd.DataFrame(p).sort(2, ascending=False).to_csv('data/manual/bk_pattern.csv')


# build assignment fuction from labeled data using Decision Tree

In [1]:
import src.constants; reload(src.constants)
from src.constants import tag_for_pattern, feature_bk_facs

In [2]:
import pandas as pd 
path = 'data/manual/bk_pattern2category.tsv'
pat = pd.read_csv(path, sep='\t')
pat.pattern = pat.pattern.apply(eval)
pat['pattern_dict'] = pat.pattern.apply(lambda x: {tag.split('=')[0]: tag.split('=')[1] for tag in x})

In [3]:
pat.head()

,pattern,cycle_lane,is_shared,cycle_way,side_walk,bikable,description,pattern_dict
0,"(highway=cycleway,)",NaN,NaN,both,NaN,NaN,two way cycleway,{u'highway': u'cycleway'}
1,"(highway=cycleway, foot=yes)",NaN,NaN,both,NaN,NaN,two way cycleway,"{u'foot': u'yes', u'highway': u'cycleway'}"
2,"(highway=cycleway, bicycle=yes, foot=yes)",NaN,NaN,both,NaN,NaN,two way cycleway,"{u'foot': u'yes', u'bicycle': u'yes', u'highwa..."
3,"(highway=cycleway, bicycle=yes)",NaN,NaN,both,NaN,NaN,two way cycleway,"{u'bicycle': u'yes', u'highway': u'cycleway'}"
4,"(highway=cycleway, bicycle=designated, foot=yes)",NaN,NaN,both,NaN,NaN,two way cycleway,"{u'foot': u'yes', u'bicycle': u'designated', u..."


In [4]:
for a in tag_for_pattern:
    pat[a] = pat.pattern_dict.apply(lambda x: x.get(a, ''))

In [14]:
import src.decision_tree; reload(src.decision_tree)
from src.decision_tree import *
trees = []
accrs = []
for y in feature_bk_facs:
    var_names = tag_for_pattern+[y]
    d = pat[var_names].fillna('').values
    tree = build_tree(d, var_names, 8,1)
    trees.append(tree)
    predictions = list()
    for row in d:
        prediction = predict(tree, row)
        predictions.append(prediction)
    actual = [row[-1] for row in d]
    accuracy = accuracy_metric(actual, predictions)
    accrs.append(accuracy)

accrs

[99.77924944812362, 100.0, 100.0, 98.45474613686534, 99.77924944812362]

In [15]:
import src.decision_tree; reload(src.decision_tree)
from src.decision_tree import *
with open('src/osm/bk_facs_rules.py', 'w') as f:
    for feature, tree in zip(feature_bk_facs,trees):
        print feature
        f.write('def {}_assignment(tags):\n'.format(feature))
        for row in print_py_func(tree, var_names):
            f.write(row+'\n')
        f.write('\n\n')

cycle_lane
is_shared
cycle_way
side_walk
bikable


In [7]:
import src.osm.bk_facs_rules;reload(src.osm.bk_facs_rules)
from src.osm.bk_facs_rules import *

In [8]:
print (pat.apply(cycle_lane_assignment, axis=1)==pat.cycle_lane.fillna('unk')).value_counts().to_dict()
print (pat.apply(is_shared_assignment, axis=1)==pat.is_shared.fillna('unk')).value_counts().to_dict()
print (pat.apply(cycle_way_assignment, axis=1)==pat.cycle_way.fillna('unk')).value_counts().to_dict()
print (pat.apply(side_walk_assignment, axis=1)==pat.side_walk.fillna('unk')).value_counts().to_dict()
print (pat.apply(bikable_assignment, axis=1)==pat.bikable.fillna('unk')).value_counts().to_dict()

{False: 1, True: 452}
{True: 453}
{True: 453}
{False: 7, True: 446}
{False: 1, True: 452}


In [9]:
pr = pd.DataFrame([pat.apply(cycle_lane_assignment, axis=1),pat.cycle_lane.fillna('unk')], index=['pred','real']).T
print pr[pr.pred!=pr.real].values

pr = pd.DataFrame([pat.apply(is_shared_assignment, axis=1),pat.is_shared.fillna('unk')], index=['pred','real']).T
print pr[pr.pred!=pr.real].values

pr = pd.DataFrame([pat.apply(cycle_way_assignment, axis=1),pat.cycle_way.fillna('unk')], index=['pred','real']).T
print pr[pr.pred!=pr.real].values

pr = pd.DataFrame([pat.apply(side_walk_assignment, axis=1),pat.side_walk.fillna('unk')], index=['pred','real']).T
print pr[pr.pred!=pr.real].values

pr = pd.DataFrame([pat.apply(bikable_assignment, axis=1),pat.bikable.fillna('unk')], index=['pred','real']).T
print pr[pr.pred!=pr.real].values



[['unk' 'one']]
[]
[]
[['no' 'unk']
 ['unk' 'no']
 ['unk' 'no']
 ['no' 'both']
 ['no' 'both']
 ['unk' 'no']
 ['unk' 'no']]
[['yes' 'no']]


# TODO: manual assignment rule function

In [11]:
from src.constants import fn_osm_db_dc
from src.osm.osmdb_filter import select_bk_facs
df = select_bk_facs('data/'+fn_osm_db_dc, df=True, ot='W')

bk_ways = []
for (ot, oid), grp in df.groupby(['ot','oid']):
    tags = {key:value for key, value in grp[['key','value']].values}
    way = {'id':oid, 'tags':tags}
    bk_ways.append(way)

In [12]:
tags_ways = []
for w in bk_ways:
    tags_ways.append([w['id']]+[w['tags'].get(t,None) for t in tag_for_pattern])


In [13]:
import pandas as pd

vectors = pd.DataFrame(tags_ways, columns=['id']+tag_for_pattern)

def pattern(row):
    a =row.dropna()[1:]
    return tuple(['{}={}'.format(k,v) for k,v in zip(a.index.tolist(), a.tolist())])


vectors['pattern']=vectors.apply(pattern,axis=1)

In [26]:
def cycle_lane_assignment(tags):
    highway = tags['highway']
    cycleway = tags['cycleway']
    oneway = tags['oneway']
    cycleway_right = tags['cycleway:right']
    cycleway_left = tags['cycleway:left']
    if highway=='cycleway':
        return None
    if cycleway in ['lane', 'shared_lane', 'opposite_lane']:
        if oneway=='yes':
            return 'one'
        return 'both'
#     elif cycleway_right=='' and cycleway_left=='':
#         if bicycle!='':
#             return 
        
    

In [14]:
(pat.apply(cycle_lane_assignment, axis=1)==pat.cycle_lane.fillna('unk')).value_counts()

True     452
False      1
dtype: int64

In [15]:
pat[pat.cycleway!='lane'].cycleway.value_counts()

                 372
shared_lane        8
track              7
no                 6
opposite_lane      2
segregated         1
dtype: int64

In [16]:
mask = (pat['cycleway']=='lane') & (pat['oneway']=='yes') & (pat['highway']=='cycleway')
mask = (pat['cycleway:left']!='') & (pat['cycleway:right']=='') & (pat['highway']!='cycleway') & (pat['cycleway']!='lane')
mask = (pat['cycleway:left']=='opposite_lane')
mask = (pat['bicycle']!='') & (pat['cycleway:right']=='') & (pat['highway']!='cycleway') & (pat['cycleway']!='lane')
print pat[mask][['cycle_way','cycle_lane', 'is_shared','side_walk','bikable']].to_dict('list')
vectors[vectors.pattern.isin(pat[mask].pattern.values)]

{'cycle_lane': [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 'both', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 'one', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 'bikable': ['yes', 'yes', 'yes', 'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'no', 'no', 'no', 'no', 'no', nan, 'yes', 'yes', 'yes', 'yes', 'no', 'no', 'no', 'yes', 'no', 'yes', 'no', 'no', 'yes', 'yes', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'yes', 'yes', 'no', 'no', 'no', 'no', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'n

,id,highway,cycleway,cycleway:left,cycleway:right,cycleway:both,oneway:bicycle,bicycle,bicycle:lanes,bicycle:backward,amenity,foot,sidewalk,oneway,lanes,pattern
1,5973117,footway,None,None,None,None,None,no,None,None,None,None,None,None,None,"(highway=footway, bicycle=no)"
10,6051295,None,None,None,None,None,None,yes,None,None,None,yes,None,None,None,"(bicycle=yes, foot=yes)"
22,6051737,None,None,None,None,None,None,yes,None,None,None,yes,None,None,None,"(bicycle=yes, foot=yes)"
55,6052535,None,None,None,None,None,None,yes,None,None,None,yes,None,None,None,"(bicycle=yes, foot=yes)"
57,6052574,None,None,None,None,None,None,no,None,None,None,None,None,None,None,"(bicycle=no,)"
66,6052876,footway,None,None,None,None,None,yes,None,None,None,None,None,None,None,"(highway=footway, bicycle=yes)"
73,6053265,None,None,None,None,None,None,yes,None,None,None,yes,None,None,None,"(bicycle=yes, foot=yes)"
405,6058204,None,None,None,None,None,None,yes,None,None,None,yes,None,None,None,"(bicycle=yes, foot=yes)"
470,6059749,None,None,None,None,None,None,yes,None,None,None,yes,None,None,None,"(bicycle=yes, foot=yes)"
492,6059971,None,None,None,None,None,None,yes,None,None,None,yes,None,None,None,"(bicycle=yes, foot=yes)"


In [17]:
vectors[vectors.id==112192545]

,id,highway,cycleway,cycleway:left,cycleway:right,cycleway:both,oneway:bicycle,bicycle,bicycle:lanes,bicycle:backward,amenity,foot,sidewalk,oneway,lanes,pattern
2102,112192545,None,None,shared_lane,lane,None,None,None,None,None,None,None,both,None,None,"(cycleway:left=shared_lane, cycleway:right=lan..."
